In [1]:
# Naughty hack...
import sys
!{sys.executable} -m pip install pytest ipytest


[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import pytest
import ipytest

from skimage import data, io
from skimage.color import rgb2gray
from skimage.transform import resize
from scipy import ndimage

from linear_system_super_resolution import *

ipytest.autoconfig()

# Unit testing support functions
def sum_of_submatrix(matrix, x, y, x_width, y_width):
    return np.sum(matrix[y:y+y_width, x:x+x_width])

### Unit test development code block

### Unit tests

In [3]:
# Test checks to see if decimation via matrix operation is equivalent to decimation by localised summation of a 2D numpy matrix
def test_decimate_matrix():
    high_res_num_pixels = 32
    low_res_num_pixel = 16
    downsample = high_res_num_pixels // low_res_num_pixel
    d_mat = decimation_matrix(high_res_num_pixels, downsample)
    high_res_img = resize(rgb2gray(data.astronaut()), (high_res_num_pixels, high_res_num_pixels), anti_aliasing=True)
    low_res_img = d_mat @ high_res_img.flatten()
    low_res_sampling_points_1d = np.arange(low_res_num_pixel) * downsample
    low_res_sampling_points_2d = np.array(np.meshgrid(low_res_sampling_points_1d, low_res_sampling_points_1d)).T.reshape(-1,2)
    sums = np.vectorize(sum_of_submatrix, excluded=[0])(high_res_img, low_res_sampling_points_2d[:, 1], low_res_sampling_points_2d[:, 0], downsample, downsample)
    assert np.array_equal(sums, low_res_img)

In [4]:
def test_decimate_matrix_transpose():
    assert False

In [5]:
# Test checks to see if convolution of a 2D filter (i.e., gaussian) via matrix operator is equivalent to a standard image convolution
def test_convolution_via_matrix():
    image_dimension = 32
    img = resize(rgb2gray(data.astronaut()), (image_dimension, image_dimension), anti_aliasing=True)
    gauss = gaussian_2d(11)
    img_smoothed = ndimage.convolve(img, gauss, mode='constant')
    smoothing_matrix = generate_h_matrix(image_dimension, gauss)
    img_smoothed_via_matrix =  (smoothing_matrix @ img.flatten()).reshape(image_dimension, image_dimension)
    assert np.allclose(img_smoothed, img_smoothed_via_matrix)

In [6]:
# Test checks to see if convolution of a 2D filter (i.e., gaussian) via matrix operator (tranposed) is equivalent to a standard image convolution
def test_convolution_via_matrix_transpose():
    high_res_num_pixels = 32
    img = resize(rgb2gray(data.astronaut()), (high_res_num_pixels, high_res_num_pixels), anti_aliasing=True)
    gauss = gaussian_2d(11)
    img_smoothed = ndimage.convolve(img, gauss, mode='constant')
    smoothing_matrix = generate_h_matrix(high_res_num_pixels, gauss)
    img_smoothed_via_matrix =  (smoothing_matrix.T @ img.flatten()).reshape(high_res_num_pixels, high_res_num_pixels)
    assert np.allclose(img_smoothed, img_smoothed_via_matrix)

In [7]:
# Test checks to see if independant matrix operations (convolution followed by decimation) is equivalent to a combined conv-deci matrix operator
def test_decimate_and_convolution_matrix():
    high_res_num_pixels = 128
    low_res_num_pixel = 32
    downsample = high_res_num_pixels // low_res_num_pixel
    high_res_img = resize(rgb2gray(data.astronaut()), (high_res_num_pixels, high_res_num_pixels), anti_aliasing=True)
    gauss = gaussian_2d(21)
    
    # Independant convolution operation
    smoothing_matrix = generate_h_matrix(high_res_num_pixels, gauss)
    high_res_img_smoothed = smoothing_matrix @ high_res_img.flatten()
    
    # Independant decimation operation
    d_mat = decimation_matrix(high_res_num_pixels, downsample)
    low_res_img_smoothed = d_mat @ high_res_img_smoothed
    
    # Combined conv-deci operation
    dh_mat = generate_dh_matrix_batched(gauss, high_res_num_pixels, downsample, low_res_num_pixel**2)
    low_res_img_smoothed_combo = dh_mat @ high_res_img.flatten()
    
    assert np.allclose(low_res_img_smoothed, low_res_img_smoothed_combo)

In [8]:
def test_decimate_and_convolution_matrix_transpose():
    assert False

In [9]:
ipytest.run()

.F...F                                                                                       [100%]
============================================= FAILURES =============================================
__________________________________ test_decimate_matrix_transpose __________________________________

    def test_decimate_matrix_transpose():
>       assert False
E       assert False

/tmp/ipykernel_3662/720013556.py:2: AssertionError
__________________________ test_decimate_and_convolution_matrix_transpose __________________________

    def test_decimate_and_convolution_matrix_transpose():
>       assert False
E       assert False

/tmp/ipykernel_3662/3466232387.py:2: AssertionError
===================================== short test summary info ======================================
FAILED t_d4f8798772f94c23a21b219610bed503.py::test_decimate_matrix_transpose - assert False
FAILED t_d4f8798772f94c23a21b219610bed503.py::test_decimate_and_convolution_matrix_transpose - assert False
2 fai

<ExitCode.TESTS_FAILED: 1>